In [1]:
from z3 import *

In [2]:
s = Solver()

In [3]:
def split(l, n):
    return [l[j:j+n] for j in range(0,len(l),n)]

In [4]:
print("\n".join(", ".join(l) for l in split([n for n in dir(s)], 6)))

__class__, __copy__, __deepcopy__, __del__, __delattr__, __dict__
__dir__, __doc__, __enter__, __eq__, __exit__, __format__
__ge__, __getattribute__, __gt__, __hash__, __iadd__, __init__
__init_subclass__, __le__, __lt__, __module__, __ne__, __new__
__reduce__, __reduce_ex__, __repr__, __setattr__, __sizeof__, __str__
__subclasshook__, __weakref__, _repr_html_, add, append, assert_and_track
assert_exprs, assertions, backtrack_level, check, consequences, ctx
cube, cube_vars, dimacs, from_file, from_string, help
import_model_converter, insert, interrupt, model, next, non_units
num_scopes, param_descrs, pop, proof, push, reason_unknown
reset, root, set, sexpr, solver, statistics
to_smt2, trail, trail_levels, translate, units, unsat_core
use_pp


In [5]:
help(s.assertions)

Help on method assertions in module z3.z3:

assertions() method of z3.z3.Solver instance
    Return an AST vector containing all added constraints.
    
    >>> s = Solver()
    >>> s.assertions()
    []
    >>> a = Int('a')
    >>> s.add(a > 0)
    >>> s.add(a < 10)
    >>> s.assertions()
    [a > 0, a < 10]



In [6]:
help(s.add)

Help on method add in module z3.z3:

add(*args) method of z3.z3.Solver instance
    Assert constraints into the solver.
    
    >>> x = Int('x')
    >>> s = Solver()
    >>> s.add(x > 0, x < 2)
    >>> s
    [x > 0, x < 2]



In [7]:
x = Int('x')

In [8]:
s.add(x>9)

In [9]:
s.assertions()

[x > 9]

In [12]:
v =s.check()
m = s.model()
print ("x = %s" % m[x])

x = None


In [11]:
x = Bool('x')

In [13]:
s.add(Implies(x, False))

In [14]:
s.assertions()

[x > 9, Implies(x, False)]

In [15]:
v =s.check()
m = s.model()
print ("x = %s" % m[x])

x = False


In [16]:
False > 9

False

In [39]:
dir(mod

['__class__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_repr_html_',
 'r']